In [1]:
# from google.colab import files

# uploaded = files.upload()

!mkvirtualenv keras_tf -p python3
!workon keras_tf
!pip install --upgrade tensorflow

/bin/sh: 1: mkvirtualenv: not found
/bin/sh: 1: workon: not found
Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (1.8.0)
Requirement not upgraded as not directly required: six>=1.10.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (1.11.0)
Requirement not upgraded as not directly required: gast>=0.2.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (0.2.0)
Requirement not upgraded as not directly required: protobuf>=3.4.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (3.5.2.post1)
Requirement not upgraded as not directly required: numpy>=1.13.3 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (1.14.3)
Requirement not upgraded as not directly required: astor>=0.6.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (0.6.2)
Requirement not upgraded as not directly required: absl-py>=0.1.6 in /usr/local/lib/python3.6/dist-packages (from tensorflow) (0.2.2)
Requirement not upgraded as not direc

In [2]:
import tensorflow as tf
print("GPU Enabled in TensorFlow Backend @",tf.test.gpu_device_name())
def authenticate():
  !pip install -U -q PyDrive
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  return drive

GPU Enabled in TensorFlow Backend @ 


In [0]:
def download(folderID, fname, can_print_contents=False):
  !pip install -U -q pprint
  import pprint
  drive = authenticate()
  files = (drive.ListFile({'q': "'"+ folderID +"' in parents and trashed=false"}).GetList())
  fdict = {file['title']:file['id'] for file in files}
  if can_print_contents:
    pprint.PrettyPrinter(indent=4).pprint(fdict)
  train_downloaded = drive.CreateFile({'id': fdict[fname]})
  train_downloaded.GetContentFile(fname)

In [4]:
FOLDERID =  '1VAx868PjZxNybXyOEG18Ka9cXy5hZiet'

download(FOLDERID, 'sentclf_train.csv',True)
download(FOLDERID, 'sentclf_test_data.csv')


{   'Copy of conv.ipynb': '1rN0ZlEdpAhIMtKNNbDy6Zlu6Nixz3aIs',
    'movie_review.ipynb': '1RLjoVCTGDT5Olm1MDdNhJAHXfMXXT31-',
    'sentclf_test_data.csv': '110GGIA6GKiAio9_OgJAT6tMUQpUn6QMv',
    'sentclf_train.csv': '1HrLM_yOUrSmq-pxKAYcqzR6o6jPVGCiy',
    'shef_sentiment.ipynb': '1LVMcqd-MpHY0DL2Ius9DFSAzuBTfwArn'}


In [0]:
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd 
from nltk.corpus import stopwords
from nltk import pos_tag
from sklearn.feature_extraction.text import CountVectorizer


In [4]:
import pandas as pd
data=pd.read_csv('sentclf_train.csv',encoding='utf-8')
test_file=pd.read_csv("sentclf_test_data.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:

data.drop(data.index[273514],axis=0,inplace=True)
data=data.reset_index()

In [0]:
IDs=test_file['ID']
test_file.drop('ID',axis=1,inplace=True)

In [7]:

test_file

,text
0,"Great mandi place, haven't seen many in the GT..."
1,Just had my pizza delivered by the best delive...
2,"The owners, Greg and Tammy promptly contacted ..."
3,One week ago:\nSigned papers for a storage uni...
4,Prior to taking in the Amon Amarth concert at ...
5,The food here is delicious! It's not your typi...
6,Don't listen to the 1 star reviews. They obvio...
7,Restaurant was very clean. People were very fr...
8,Horrible don't waste your time. It had no flav...
9,We hired Josh to help our son prep for the ACT...


In [0]:
# from nltk.tokenize import sent_tokenize, word_tokenize
# import pandas as pd 
# from nltk.corpus import stopwords
# from nltk import pos_tag
# from sklearn.feature_extraction.text import CountVectorizer
# import nltk

In [11]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> x

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [9]:
documents=[]
for indx in range(len(data)):
    documents.append(word_tokenize(str(data['text'][indx])))
    if indx%1000==0:
      print(indx)
for indx in range(len(test_file)):
  documents.append(word_tokenize(str(test_file['text'][indx])))

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


181000
182000
183000
184000
185000
186000
187000
188000
189000
190000
191000
192000
193000
194000
195000
196000
197000
198000
199000
200000
201000
202000
203000
204000
205000
206000
207000
208000
209000
210000
211000
212000
213000
214000
215000
216000
217000
218000
219000
220000
221000
222000
223000
224000
225000
226000
227000
228000
229000
230000
231000
232000
233000
234000
235000
236000
237000
238000
239000
240000
241000
242000
243000
244000
245000
246000
247000
248000
249000
250000
251000
252000
253000
254000
255000
256000
257000
258000
259000
260000
261000
262000
263000
264000
265000
266000
267000
268000
269000
270000
271000
272000
273000
274000
275000
276000
277000
278000
279000
280000
281000
282000
283000
284000
285000
286000
287000
288000
289000
290000
291000
292000
293000
294000
295000
296000
297000
298000
299000
300000
301000
302000
303000
304000
305000
306000
307000
308000
309000
310000
311000
312000
313000
314000
315000
316000
317000
318000
319000
320000
321000
322000
323000

349000
350000
351000
352000
353000
354000
355000
356000
357000
358000
359000
360000
361000
362000
363000
364000
365000
366000
367000
368000
369000
370000
371000
372000
373000
374000
375000
376000
377000
378000
379000
380000
381000
382000
383000
384000
385000
386000
387000
388000
389000
390000
391000
392000
393000
394000
395000
396000
397000
398000
399000


In [42]:
len(documents)

500000

In [0]:
# from nltk.corpus import wordnet
# def get_simple_pos(tag):
#     if tag.startswith('J'):
#         return wordnet.ADJ
#     elif tag.startswith('V'):
#         return wordnet.VERB
#     elif tag.startswith('N'):
#         return wordnet.NOUN
#     elif tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return wordnet.NOUN

In [0]:
# from nltk.stem import WordNetLemmatizer
# lemmatizer=WordNetLemmatizer()

In [0]:
# from nltk.corpus import stopwords
# import string
# stops=set(stopwords.words('english'))
# punctuation=list(string.punctuation)
# stops.update(punctuation)
# len(stops)

In [0]:
# stops

In [0]:
# def clean_review(words):
#     output_words=[]
# #     passing words as it is important ex. capital means proper noun
#     for w in words:
#         if w.lower() not in stops:
#             pos=pos_tag((w))
#             clean_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
#             output_words.append(clean_word.lower())
#     return output_words
  

In [0]:
# data.shape

In [0]:
# # documents=[ (clean_review(documents[indx]),data.label[indx]) for indx in range(len(documents))]
# documentts=[]
# for indx in range(len(documents)):
#     documentts.append((clean_review(documents[indx])))
#     if indx%1000==0:
#       print(indx)


In [43]:
len(documents)

500000

In [0]:
# len(documentts)

In [44]:
category=[data['label'][indx] for indx in range(len(data))]
len(category)

400000

In [0]:
text_documents=[ " ".join(document) for document in documents ]


In [46]:
len(text_documents)

500000

In [0]:
#  RUN TILL HERE

In [0]:
# import pickle

In [0]:
# pickle_out=open("sentimentall.pickle","wb")

# pickle.dump(text_documents,pickle_out)   
# pickle_out.close()

In [0]:
# pickle_in=open("sentimentall.pickle","rb")
# text_documents= pickle.load(pickle_in)  


In [0]:
# text_documents

In [25]:
len(text_documents)


500000

In [26]:
category

[1,
 2,
 2,
 1,
 1,
 0,
 2,
 0,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 1,
 0,
 0,
 2,
 2,
 0,
 1,
 1,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 2,
 0,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 0,
 1,
 1,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 0,
 2,
 2,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 2,
 2,
 1,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 1,
 2,
 2,
 1,
 0,
 1,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 0,
 0,


In [0]:
# x_train=text_documents[0:400000]

In [0]:
# x_test=text_documents[400000:]

In [0]:
# len(x_train)

In [0]:
# len(x_test)

In [0]:
# y_train=category

In [0]:
from sklearn.model_selection import train_test_split


In [0]:
x_train, x_test,y_train, y_test=train_test_split(text_documents[0:400000], category,random_state=1)

In [0]:
count_vec=CountVectorizer(max_features=3000,ngram_range=(1,2),max_df=0.97,min_df=0.05)
x_train_features=count_vec.fit_transform(x_train)
x_train_features.todense()

In [52]:
len(count_vec.get_feature_names())

290

In [0]:
x_test_features=count_vec.transform(x_test)

In [37]:
x_test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 1],
        [0, 1, 0, ..., 0, 0, 3],
        [0, 0, 0, ..., 0, 0, 0]])

In [0]:
# Naive Bayes

In [51]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_features, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [40]:
clf.score( x_test_features, y_test)

0.76501

In [0]:
# import numpy as np

# sentim_NB = pd.DataFrame({'ID':IDs , "label": clf.predict(x_test_features)})
# sentim_NB.to_csv('senti_NB.csv', index=False)

In [0]:
#svc

In [0]:
from sklearn.svm import SVC

In [1]:
svc=SVC(C=100)
svc.fit(x_train_features, y_train)

NameError: ignored

In [64]:
svc.score(x_test_features, y_test)

0.87048

In [0]:
# import numpy as np

# senti_svc = pd.DataFrame({'ID':IDs , "label": svc.predict(x_test_features)})
# senti_svc.to_csv('senti_svc.csv', index=False)

In [0]:
# 

In [0]:
# from sklearn.ensemble import RandomForestClassifier

In [0]:
# clf = RandomForestClassifier()

In [0]:
# clf.fit(x_train_features, y_train)
# clf.score(x_test_features, y_test)